In [4]:
from langchain.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
def load_docs(directory_path):
    loader = DirectoryLoader(directory_path, glob="**/*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
doc = load_docs("data")

In [4]:
print(doc[10].metadata)
print(doc[10].page_content)

{'source': 'data\\a16-ahn.pdf', 'page': 10}
16:10 J. Ahn et al.
Our system uses a commercial image labeling Web service, called IQEngines
[IQEngines 2013], to determine the user’s initial starting position when using our
grocery shopping application. IQEngines uses a combination of computer vision and
crowdsourcing to tag a photo with a label describing the content of the image. For
example, an image of a box of Frosted Cheerios cereal might be labeled “General Mills
Frosted Cheerios”. When an image is submitted to IQEngines, the image is ﬁrst pro-
cessed by a computer vision system in an effort to provide an accurate label. If the
computer vision system cannot identify the image, then IQEngines passes the image to
its crowdsourcing network for analysis and tagging. According to IQEngines, the time
to return a label for an image varies from a few seconds for the computer vision sys-
tem, to a few minutes for the crowdsourcing system. To ensure fast image labeling in
our experiments, we

In [5]:
def make_chunks(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        add_start_index=True
    )

    chunks = text_splitter.split_documents(docs)

    return chunks

In [6]:
chunks = make_chunks(doc)
chunks[10]

Document(metadata={'source': 'data\\a16-ahn.pdf', 'page': 3, 'start_index': 0}, page_content='Supporting Healthy Grocery Shopping via Mobile Augmented Reality 16:3\nAR tag grows in size. The tags when clicked reveal nutritional information about the\nproduct. The tags are also colored, for example, green to indicate products that are\nnutritionally preferable (e.g., low-calorie, gluten-free), and red to indicate products to\navoid (e.g., high cholesterol or peanut content). Further, shoppers can specify health\nproﬁles which may impact their food purchase choices, such as weight control, heart\ndisease, food allergies, etc. The recommended products shown via AR tags will then\nchange depending on what health condition/concern is indicated by the user. We believe\nour system is the ﬁrst to integrate augmented reality tagging and pedometry-based\nlocalization with a back-end server to provide health-based grocery recommendations\nat the point-of-purchase. We evaluated the effectiveness o

In [7]:
chunks[11]

Document(metadata={'source': 'data\\a16-ahn.pdf', 'page': 3, 'start_index': 754}, page_content='localization with a back-end server to provide health-based grocery recommendations\nat the point-of-purchase. We evaluated the effectiveness of our system in a real gro-\ncery store aisle with 15 actual grocery shopping subjects to determine how easy and\nfast the subjects reported it was to locate healthy food products and avoid unhealthy\nones, using AR tagging with our application. We also evaluated our application’s func-\ntionality and performance by analyzing data we collected from 104 online application\ndemonstration/survey participants.\n2. RELATED WORK\nAugmented reality has been recently applied in the mobile health arena in a variety\nof applications. For example, AR tags are overlaid in a home environment to pro-\nvide instructions to the elderly for tasks like taking medication, cooking, washing,\netc. [Herv´as et al. 2011]. TriggerHunter is an AR-based game that overlays tags

In [7]:
# from langchain.embeddings import HuggingFaceEmbeddings,OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceInferenceAPIEmbeddings
# import chromadb
# from chromadb.utils import embedding_functions
from langchain_chroma import Chroma
from chromadb import Client
from chromadb.config import Settings



In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

HF_TOKEN = os.getenv('HF_TOKEN')

In [9]:
def save_embeddings(chunks, embedding_function,CHROMA_PATH="chroma_db"):
    chroma = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_function,
        persist_directory=CHROMA_PATH,
        collection_name="my_collection"
    )
    # chroma.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


In [10]:

# Define Chroma database path
CHROMA_PATH = "chroma_db"

# Initialize Hugging Face embedding
hugging_face_ef = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN,
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [ ]:
# save_embeddings(chunks, hugging_face_ef,CHROMA_PATH)

Saved 162 chunks to chroma_db.


C:\Users\Dell\AppData\Local\Temp\ipykernel_22236\452922040.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma.persist()


In [11]:
def get_top_docs(query,k=4):
    # load db
    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=hugging_face_ef,
        collection_name="my_collection"
    )
        
    # search db
    results = db.similarity_search_with_relevance_scores(query,k=k)
    
    return results
        

In [19]:

# get query input
query = input("Enter your query: ")

result = get_top_docs(query)

In [20]:
result

[(Document(metadata={'page': 2, 'source': 'data\\a16-ahn.pdf', 'start_index': 2301}, page_content='by the mobile camera and overlays the result on top of the text [WordLens 2014]; and\nmany more.\nA prototype of our augmented reality mobile grocery shopping application is shown\nin Figure 1. As the user pans and walks up and down a grocery store aisle, the AR\ntags corresponding to highlighted products will change based on what products the\nuser is facing. As a user walks towards an item along the aisle, its corresponding\nACM Trans. Multimedia Comput. Commun. Appl., Vol. 12, No. 1s, Article 16, Publication date: October 2015.'),
  0.5298927030103356),
 (Document(metadata={'page': 1, 'source': 'data\\review_paper.pdf', 'start_index': 1613}, page_content='have accumulated from 104 participants of our online demonstration/survey.\nReview\nThere are some works regarding shopping assistant that has done by other researchers. However, there are\nstill many challenges and limitation exist i

In [ ]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-1.5-flash")

d:\Coding\RAG using langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
def generate_response(query):
    PROMPT_TEMPLATE = """
Answer the question given based only on the context given below.
context:
{context}

---
based on the context above answer the following query:
{query}
"""
    top_3_results = get_top_docs(query,k=3)
    # if top result is not releavent that is score less than 0.6
    if top_3_results[0][1] < 0.6:
        print("Query is not relevant to the provided material. So the output might not be accurate.")
    
    context = "\n".join([result[0].page_content for result in top_3_results])
    prompt = PROMPT_TEMPLATE.format(context=context,query=query)
    response = model.generate_content(prompt)
    print("Query:",query)
    print("Response:",response.text)
    print("Sources:" + ",".join([result[0].metadata['source'] + f" [Page:{int(result[0].metadata['page']) + 1}]" if 'page' in result[0].metadata else "" for result in top_3_results]))

    

In [22]:

generate_response(input())


Query is not relevant to the provided material. So the output might not be accurate.
Query: how ar affect consumer behaivour
Response: Based on the provided text, Augmented Reality (AR) shows potential for influencing consumer behavior toward healthier food choices by offering personalized suggestions and information about products.  However, studies reveal that its effectiveness is lower than traditional methods like Front-of-Package (FOP) labeling.  While AR apps may increase intentions to buy healthier products in hypothetical situations, this doesn't always translate to actual behavior in real-world shopping scenarios.  The complexity and ease of use of AR applications are significant hurdles to overcome to maximize their impact on consumer behavior.

Sources:data\review_paper.pdf[Page:7],data\review_paper.pdf[Page:6],data\review_paper.pdf[Page:6]
